![](https://pm4py.fit.fraunhofer.de/static/assets/images/pm4py-site-logo-padded.png)

# <b>Motivation</b>
Traditional event logs, used by mainstream process mining techniques, require the events to be related to a case. A case is a set of events for a particular purpose. A case notion is a criteria to assign a case to the events.

However, in real processes this leads to two problems:

* If we consider the Order-to-Cash process, an order could be related to many different deliveries. If we consider the delivery as case notion, the same event of Create Order needs to be replicated in different cases (all the deliveries involving the order). This is called the <b>convergence</b> problem.
* If we consider the Order-to-Cash process, an order could contain different order items, each one with a different lifecycle. If we consider the order as case notion, several instances of the activities for the single items may be contained in the case, and this make the frequency/performance annotation of the process problematic. This is called the <b>divergence</b> problem.

<b>Object-centric event logs</b> relax the assumption that an event is related to exactly one case. Indeed, an event can be related to different objects of different object types.

Essentially, we can describe the different components of an object-centric event log as:

* <b>Events</b>, having an identifier, an activity, a timestamp, a list of related objects and a dictionary of other attributes.
* <b>Objects</b>, having an identifier, a type and a dictionary of other attributes.
* <b>Attribute names</b>, e.g. the possible keys for the attributes of the event/object attribute map.
* <b>Object types</b>, e.g. the possible types for the objects.
<br></br>


# <b>Supported Formats</b>
Several historical formats (OpenSLEX, XOC) have been proposed for the storage of object-centric event logs. In particular, the [OCEL standard](http://www.ocel-standard.org/) proposes lean and intercompatible formats for the storage of object-centric event logs. These include:

* <b>XML-OCEL</b>: a storage format based on XML for object-centric event logs. An example of XML-OCEL event log is reported [here](https://github.com/pm4py/pm4py-core/blob/release/tests/input_data/ocel/example_log.xmlocel).
* <b>JSON-OCEL</b>: a storage format based on JSON for object-centric event logs. An example of JSON-OCEL event log is reported [here](https://github.com/pm4py/pm4py-core/blob/release/tests/input_data/ocel/example_log.jsonocel).

Among the commonalities of these formats, the event/object identifier is <b>ocel:id</b>, the activity identifier is <b>ocel:activity</b>, the timestamp of the event is <b>ocel:timestamp</b>, the type of the object is <b>ocel:type</b>. Moreover, the list of related objects for the events is identified by <b>ocel:omap</b>, the attribute map for the events is identified by <b>ocel:vmap</b>, the attribute map for the objects is identified by <b>ocel:ovmap</b>.

Ignoring the attributes at the object level, we can also represent the object-centric event log in a CSV format (an example is reported [here](https://github.com/pm4py/pm4py-core/blob/release/tests/input_data/ocel/example_log.csv)). There, a row represent an event, where the event identifier is <b>ocel:eid</b>, and the related objects for a given type OTYPE are reported as a list under the voice <b>ocel:type:OTYPE</b>.
<br></br>


# <b>Topics:</b>

## <b>1. Setup</b>

In [1]:
import pm4py
from pm4py.algo.discovery.ocel.interleavings import algorithm as interleavings_discovery
from pm4py.algo.merging.case_relations import algorithm as case_relations_merging
from pm4py.visualization.ocel.interleavings import visualizer as interleavings_visualizer
from pm4py.algo.discovery.ocel.link_analysis import algorithm as link_analysis
from pm4py.algo.transformation.ocel.features.objects import algorithm
from pm4py.algo.transformation.ocel.features.events import algorithm
from pm4py.algo.transformation.ocel.graphs import (object_interaction_graph,
                                                   object_descendants_graph,
                                                   object_inheritance_graph,
                                                   object_cobirth_graph,
                                                   object_codeath_graph)

from pm4py.objects.ocel.util import log_ocel


import pandas as pd
from src import SRC_DIR

import warnings
warnings.filterwarnings("ignore")

<br></br>
## <b>2. Importing/Export OCELs</b>

In [2]:
# For all the supported formats, an OCEL event log can be read by doing:

ocel = pm4py.read_ocel(str(SRC_DIR / 'Datasets' / 'Example' / 'Recruiting' / 'recruiting.jsonocel'))

In [3]:
# An OCEL can also be exported easily by doing (ocel is assumed to be an object-centric event log):

pm4py.write_ocel(ocel, str(SRC_DIR / 'Datasets' / 'Example' / 'Recruiting' / 'recruiting.jsonocel'))

<br></br>
## <b>3. Basic Statistics on OCELs</b>

In [5]:
# The simplest way of obtaining some statistics on OCELs is by doing the print of the OCEL object:

print(ocel)

Object-Centric Event Log (number of events: 6980, number of objects: 1505, number of activities: 16, number of object types: 6, events-objects relationships: 21961)
Activities occurrences: {'submit application': 916, 'assign recruiter': 916, 'first screening': 916, 'send rejection': 781, 'check references': 729, 'consult manager': 729, 'assign vacancy': 458, 'invite for interview': 446, 'conduct interview': 446, 'open vacancy': 140, 'close vacancy for new applications': 140, 'make job offer': 135, 'offer accepted and hired': 89, 'close vacancy (no hire)': 51, 'job offer declined': 46, 'change manager': 42}
Object types occurrences (number of objects): {'applications': 916, 'applicants': 288, 'vacancies': 140, 'offers': 135, 'recruiters': 20, 'managers': 6}
Please use <THIS>.get_extended_table() to get a dataframe representation of the events related to the objects.


In [6]:
# The retrieval of the object types contained in the event log can be otained doing:

pm4py.ocel_get_object_types(ocel)

['vacancies', 'managers', 'applicants', 'applications', 'recruiters', 'offers']

In [16]:
# The retrieval of a dictionary containing the set of activities for each object type can be obtained using the command on the right:

pm4py.ocel_object_type_activities(ocel)

{'applications': {'assign recruiter',
  'assign vacancy',
  'check references',
  'conduct interview',
  'consult manager',
  'first screening',
  'invite for interview',
  'job offer declined',
  'make job offer',
  'offer accepted and hired',
  'send rejection',
  'submit application'},
 'recruiters': {'assign recruiter',
  'check references',
  'conduct interview',
  'consult manager',
  'first screening',
  'invite for interview',
  'job offer declined',
  'make job offer',
  'offer accepted and hired',
  'send rejection'},
 'vacancies': {'assign vacancy',
  'change manager',
  'close vacancy (no hire)',
  'close vacancy for new applications',
  'conduct interview',
  'invite for interview',
  'job offer declined',
  'make job offer',
  'offer accepted and hired',
  'open vacancy',
  'submit application'},
 'managers': {'change manager',
  'conduct interview',
  'consult manager',
  'make job offer',
  'open vacancy'},
 'applicants': {'conduct interview',
  'invite for interview',


In [28]:
# It is possible to obtain for each event identifier and object type the number of related objects to the event:

for item in pm4py.ocel_objects_ot_count(ocel).items():
    print(item)
    break

('1', Counter({'vacancies': 1, 'managers': 1}))


In [20]:
# The temporal summary is a table in which the different timestamps are reported along with the set of activities happening in a given time and the objects involved

pm4py.ocel_temporal_summary(ocel).head(5)

,ocel:timestamp,ocel:activity,ocel:oid
0,2019-05-20 12:26:57+00:00,"{open vacancy, submit application}","{Vacancy[550001] - Manager, Application[770001..."
1,2019-05-20 13:38:13+00:00,{submit application},"{Toon Jansen, Application[770002]}"
2,2019-05-20 15:00:46+00:00,{submit application},"{Jorge Li, Application[770003]}"
3,2019-05-20 15:45:06+00:00,{submit application},"{Lisa Jones, Application[770004]}"
4,2019-05-20 17:14:31+00:00,{submit application},"{Vacancy[550001] - Manager, Pete Jansen, Appli..."


In [29]:
# The objects summary is a table in which the different objects occurring in the log are reported along with the list of activities of the events related to the object,
# the start/end timestamps of the lifecycle, the duration of the lifecycle and the other objects related to the given object in the interaction graph.

pm4py.ocel_objects_summary(ocel).head(5)

,ocel:oid,activities_lifecycle,lifecycle_start,lifecycle_end,lifecycle_duration,interacting_objects
0,Alexander Rinke,"[open vacancy, open vacancy, open vacancy, ope...",2019-05-23 14:03:33+00:00,2020-07-30 10:30:08+00:00,37484795.0,"{Vacancy[550093] - Manager, Application[770615..."
1,Andre Brown,"[submit application, send rejection, submit ap...",2019-05-31 11:10:06+00:00,2019-09-10 13:59:23+00:00,8822957.0,"{Vacancy[550104] - Manager, Ed Keane, Vaishnav..."
2,Andre Davis,"[submit application, send rejection, submit ap...",2019-06-04 13:19:15+00:00,2019-10-14 15:10:25+00:00,11411470.0,"{Ed Hense, Dionne Geisler, Simon Keane, Applic..."
3,Andre Jansen,"[submit application, send rejection, submit ap...",2019-06-02 16:54:01+00:00,2019-07-10 17:05:47+00:00,3283906.0,"{Simon Meister, Application[770117], Vacancy[5..."
4,Andre Johnson,"[submit application, invite for interview, con...",2019-05-22 13:28:42+00:00,2019-10-10 16:23:56+00:00,12192914.0,"{Ed Keane, Application[770026], Vacancy[550018..."


<br></br>
## <b>4. Internal Data Structure</b>

pm4py has in total three Pandas dataframes:

* The <b>events</b> dataframe: this stores a row for each event. Each row contains the event identifier (<b>ocel:eid</b>), the activity (<b>ocel:activity</b>), the timestamp (<b>ocel:timestamp</b>), and the values for the other event attributes (one per column).
* The <b>objects</b> dataframe: this stores a row for each object. Each row contains the object identifier (<b>ocel:oid</b>), the type (<b>ocel:type</b>), and the values for the object attributes (one per column).
* The <b>relations</b> dataframe: this stores a row for every relation event->object. Each row contains the event identifier (<b>ocel:eid</b>), the object identifier (<b>ocel:oid</b>), the type of the related object (<b>ocel:type</b>).

These dataframes can be accessed as properties of the OCEL object (e.g., ocel.events, ocel.objects, ocel.relations), and be obviously used for any purposes (filtering, discovery).

<br></br>
### <b>4.1. The events:</b>

In [30]:
ocel.events.head()

,ocel:eid,ocel:timestamp,ocel:activity
0,1,2019-05-20 12:26:57+00:00,open vacancy
1,2,2019-05-20 12:26:57+00:00,submit application
2,3,2019-05-20 13:38:13+00:00,submit application
3,4,2019-05-20 15:00:46+00:00,submit application
4,5,2019-05-20 15:45:06+00:00,submit application


<br></br>
### <b>4.2. The objects:</b>

In [13]:
ocel.objects.head()

,ocel:oid,ocel:type
0,Vacancy[550001] - Manager,vacancies
1,Bastian Nominacher,managers
2,Stephan Taylor,applicants
3,Application[770001],applications
4,Toon Jansen,applicants


<br></br>
### <b>4.3. The relations:</b>

In [14]:
ocel.relations.head()

,ocel:eid,ocel:activity,ocel:timestamp,ocel:oid,ocel:type,ocel:qualifier
0,1,open vacancy,2019-05-20 12:26:57+00:00,Vacancy[550001] - Manager,vacancies,None
1,1,open vacancy,2019-05-20 12:26:57+00:00,Bastian Nominacher,managers,None
2,2,submit application,2019-05-20 12:26:57+00:00,Stephan Taylor,applicants,None
3,2,submit application,2019-05-20 12:26:57+00:00,Application[770001],applications,None
4,3,submit application,2019-05-20 13:38:13+00:00,Toon Jansen,applicants,None


<br></br>
## <b>5. Filtering Object-Centric Event Logs</b>

There are filters at three levels:
* Filters at the <b>event level</b> (operating first at the ocel.events structure and then propagating the result to the other parts of the object-centric log).
* Filters at the <b>object level</b> (operating first at the ocel.objects structure and then propagating the result to the other parts of the object-centric log).
* Filters at the <b>relations level</b> (operating first at the ocel.relations structure and then propagating the result to the other parts of the object-centric log).

In [31]:
ocel = pm4py.read_ocel(str(SRC_DIR / 'Datasets' / 'Example' / 'Orders' / 'example_log.jsonocel'))

<br></br>
### <b>5.1. Filter on Event Attributes:</b>

In [14]:
# 1. Filtering on the ocel:activity (the activity) attribute is reported on the right.
# 2. The positive boolean tells if to filter the events with an activity falling in the list or to filter the events NOT falling in the specified list (if positive is False)

pm4py.filter_ocel_event_attribute(ocel, "ocel:activity", ["Create Fine", "Send Fine"], positive=True)

Object-Centric Event Log (number of events: 0, number of objects: 0, number of activities: 0, number of object types: 0, events-objects relationships: 0)
Activities occurrences: {}
Object types occurrences (number of objects): {}
Please use <THIS>.get_extended_table() to get a dataframe representation of the events related to the objects.

<br></br>
### <b>5.2. Filter on Object Attributes:</b>

In [16]:
pm4py.filter_ocel_object_attribute(ocel, "ocel:type", ["order", "delivery"], positive=True)

Object-Centric Event Log (number of events: 20, number of objects: 6, number of activities: 12, number of object types: 2, events-objects relationships: 20)
Activities occurrences: {'Create Order': 3, 'Create Delivery': 3, 'Delivery Successful': 3, 'Invoice Sent': 2, 'Payment Reminder': 2, 'Confirm Order': 1, 'Delivery Failed': 1, 'Retry Delivery': 1, 'Pay Order': 1, 'Cancel Order': 1, 'Add Item to Order': 1, 'Send for Credit Collection': 1}
Object types occurrences (number of objects): {'order': 3, 'delivery': 3}
Please use <THIS>.get_extended_table() to get a dataframe representation of the events related to the objects.

<br></br>
### <b>5.3. Filter on Allowed Activities per Object Type:</b>

In [17]:
pm4py.filter_ocel_object_types_allowed_activities(ocel, {"order": ["Create Order"], "item": ["Create Order", "Create Delivery"]})

Object-Centric Event Log (number of events: 3, number of objects: 3, number of activities: 1, number of object types: 1, events-objects relationships: 3)
Activities occurrences: {'Create Order': 3}
Object types occurrences (number of objects): {'order': 3}
Please use <THIS>.get_extended_table() to get a dataframe representation of the events related to the objects.

<br></br>
### <b>5.4. Filter on the Number of Objects per Type:</b>

In [25]:
pm4py.filter_ocel_object_per_type_count(ocel, {"order": 1, "element": 2})

Object-Centric Event Log (number of events: 3, number of objects: 11, number of activities: 1, number of object types: 2, events-objects relationships: 11)
Activities occurrences: {'Create Order': 3}
Object types occurrences (number of objects): {'element': 8, 'order': 3}
Please use <THIS>.get_extended_table() to get a dataframe representation of the events related to the objects.

<br></br>
### <b>5.5. Filter on Start/End Events per Object:</b>

In [26]:
pm4py.filter_ocel_start_events_per_object_type(ocel, "order")
pm4py.filter_ocel_end_events_per_object_type(ocel, "order")

Object-Centric Event Log (number of events: 3, number of objects: 3, number of activities: 3, number of object types: 1, events-objects relationships: 3)
Activities occurrences: {'Pay Order': 1, 'Cancel Order': 1, 'Send for Credit Collection': 1}
Object types occurrences (number of objects): {'order': 3}
Please use <THIS>.get_extended_table() to get a dataframe representation of the events related to the objects.

<br></br>
### <b>5.6. Filter on Object Types:</b>

In [27]:
pm4py.filter_ocel_object_types(ocel, ['order', 'element'])

Object-Centric Event Log (number of events: 18, number of objects: 12, number of activities: 12, number of object types: 2, events-objects relationships: 31)
Activities occurrences: {'Create Order': 3, 'Create Delivery': 3, 'Invoice Sent': 2, 'Payment Reminder': 2, 'Confirm Order': 1, 'Item out of Stock': 1, 'Item back in Stock': 1, 'Pay Order': 1, 'Remove Item': 1, 'Cancel Order': 1, 'Add Item to Order': 1, 'Send for Credit Collection': 1}
Object types occurrences (number of objects): {'element': 9, 'order': 3}
Please use <THIS>.get_extended_table() to get a dataframe representation of the events related to the objects.

<br></br>
### <b>5.7. Filter on Event Identifiers:</b>

In [21]:
pm4py.filter_ocel_events(ocel, ['e1', 'e2'])

Object-Centric Event Log (number of events: 2, number of objects: 5, number of activities: 2, number of object types: 2, events-objects relationships: 6)
Activities occurrences: {'Create Order': 1, 'Confirm Order': 1}
Object types occurrences (number of objects): {'element': 4, 'order': 1}
Please use <THIS>.get_extended_table() to get a dataframe representation of the events related to the objects.

<br></br>
## <b>6. Flattening to a Traditional Log</b>

<b>Flattening</b> permits to convert an object-centric event log to a traditional event log with the specification of an object type. This allows for the application of traditional process mining techniques to the flattened log.

An example in which an event log is imported, and a flattening operation is applied on the <b>order</b> object type, is the following:

In [18]:
ocel = pm4py.read_ocel(str(SRC_DIR / 'Datasets' / 'Example' / 'Recruiting' / 'recruiting.jsonocel'))
flattened_log = pm4py.ocel_flattening(ocel, "applications")
flattened_log.head()

,ocel:eid,time:timestamp,concept:name,case:concept:name,case:ocel:type
0,2,2019-05-20 12:26:57+00:00,submit application,Application[770001],applications
1,3,2019-05-20 13:38:13+00:00,submit application,Application[770002],applications
2,4,2019-05-20 15:00:46+00:00,submit application,Application[770003],applications
3,5,2019-05-20 15:45:06+00:00,submit application,Application[770004],applications
4,6,2019-05-20 17:14:31+00:00,submit application,Application[770005],applications


<br></br>
## <b>7. Timestamp-Based Interleavings</b>

The situation in which an object-centric event log is produced directly at the extraction phase from the information systems is uncommon. Extractors for this settings are quite uncommon nowadays.

More frequent is the situation where some event logs can be extracted from the system and then their cases are related. So we can use the classical extractors to extract the event logs, and additionally extract only the relationships between the cases.

This information can be used to mine the relationships between events. In particular, the method of <b>timestamp-based interleavings</b> can be used. These consider the temporal flow between the different processes, based on the provided case relations: you can go from the left-process to the right-process, and from the right-process to the left-process.

In the following, we will assume the cases to be Pandas dataframes (with the classical pm4py naming convention, e.g. <b>case:concept:name</b>, <b>concept:name</b> and <b>time:timestamp</b>) and a case relations dataframe is defined between them (with the related cases being expressed respectively as <b>case:concept:name_LEFT</b> and <b>case:concept:name_RIGHT</b>.

In this example, we load two event logs, and a dataframe containing the relationships between them. Then, we apply the timestamp-based interleaved miner.

In [7]:
dataframe1 = pd.read_csv(str(SRC_DIR / 'Datasets' / 'Example' / 'Receipt' / 'receipt_even.csv'))
dataframe1 = pm4py.format_dataframe(dataframe1)
dataframe2 = pd.read_csv(str(SRC_DIR / 'Datasets' / 'Example' / 'Receipt' / 'receipt_odd.csv'))
dataframe2 = pm4py.format_dataframe(dataframe2)
case_relations = pd.read_csv(str(SRC_DIR / 'Datasets' / 'Example' / 'Receipt' / 'case_relations.csv'))

interleavings = interleavings_discovery.apply(dataframe1, dataframe2, case_relations)

TypeError: '<' not supported between instances of 'Timestamp' and 'str'

In [ ]:
interleavings.head()

<br></br>
The resulting interleavings dataframe will contain several columns, including for each row (that is a couple of related events, the first belonging to the first dataframe, the second belonging to the second dataframe):

* All the columns of the event (of the interleaving) of the first dataframe (with prefix <b>LEFT</b>).
* All the columns of the event (of the interleaving) of the second dataframe (with prefix <b>RIGHT</b>).
* The column <b>@@direction indicating the direction of the interleaving (with LR we go left-to-right so from the first dataframe to the second dataframe; with RL we go right-to-left, so from the second dataframe to the first dataframe).
* The columns <b>@@source_activity</b> and <b>@@target_activity</b> contain respectively the source and target activity of the interleaving.
* The columns <b>@@source_timestamp</b> and <b>@@target_timestamp</b> contain respectively the source and target timestamp of the interleaving.
* The column <b>@@left_index</b> contains the index of the event of the first of the two dataframes.
* The column <b>@@right_index</b> contains the index of the event of the second of the two dataframes.
* The column <b>@@timestamp_diff</b> contains the difference between the two timestamps (can be useful to aggregate on the time).

<br></br>
### <b>7.1. Visualization of the interleavings between the two logs</b>

In [33]:
# frequency-based
gviz_freq = interleavings_visualizer.apply(dataframe1, dataframe2, interleavings, parameters={"annotation": "frequency", "format": "svg"})
interleavings_visualizer.save(gviz_freq, output_file_path = SRC_DIR / 'Images' / 'OCPM' / '1_interleavings.svg')

NameError: name 'interleavings' is not defined

In [ ]:
# performance-based
gviz_perf = interleavings_visualizer.apply(dataframe1, dataframe2, interleavings, parameters={"annotation": "performance", "aggregation_measure": "median", "format": "png"})
interleavings_visualizer.save(gviz_perf, output_file_path = SRC_DIR / 'Images' / 'OCPM' / '2_interleavings.png')


The parameters offered by the visualization of the interleavings follows:

* <b>Parameters.FORMAT</b>: the format of the visualization (svg, png).
* <b>Parameters.BGCOLOR</b>: background color of the visualization (default: transparent).
* <b>Parameters.RANKDIR</b>: the direction of visualization of the diagram (LR, TB).
* <b>Parameters.ANNOTATION</b>: the annotation to be used (frequency, performance).
* <b>Parameters.AGGREGATION_MEASURE</b>: the aggregation to be used (mean, median, min, max).
* <b>Parameters.ACTIVITY_PERCENTAGE</b>: the percentage of activities that shall be included in the two DFGs and the interleavings visualization.
* <b>Parameters.PATHS_PERCENTAG</b>: the percentage of paths that shall be included in the two DFGs and the interleavings visualization.
* <b>Parameters.DEPENDENCY_THRESHOLD</b>: the dependency threshold that shall be used to filter the edges of the DFG.
* <b>Parameters.MIN_FACT_EDGES_INTERLEAVINGS</b>: parameter that regulates the fraction of interleavings that is shown in the diagram.

<br></br>
## <b>8. Network Analysis</b>
The classical social network analysis methods (such as the ones described in this page at the later sections) are based on the order of the events inside a case. For example, the Handover of Work metric considers the directly-follows relationships between resources during the work of a case. An edge is added between the two resources if such relationships occurs.

Real-life scenarios may be more complicated. At first, is difficult to collect events inside the same case without having convergence/divergence issues. At second, the type of relationship may also be important. Consider for example the relationship between two resources: this may be more efficient if the activity that is executed is liked by the resources, rather than disgusted.

The network analysis that we introduce in this section generalizes some existing social network analysis metrics, becoming independent from the choice of a case notion and permitting to build a multi-graph instead of a simple graph.

With this, we assume events to be linked by signals. An event emits a signal (that is contained as one attribute of the event) that is assumed to be received by other events (also, this is an attribute of these events) that follow the first event in the log. So, we assume there is an OUT attribute (of the event) that is identical to the IN attribute (of the other events).

When we collect this information, we can build the network analysis graph:

* The source node of the relation is given by an aggregation over a node_column_source attribute.
* The target node of the relation is given by an aggregation over a node_column_target attribute.
* The type of edge is given by an aggregation over an edge_column attribute.

The network analysis graph can either be annotated with frequency or performance information.

In [27]:
log = pm4py.read_xes(str(SRC_DIR / 'Datasets' / "Example/Receipt/receipt.xes"))

frequency_edges = pm4py.discover_network_analysis(log,
                                                  out_column="case:concept:name",
                                                  in_column="case:concept:name",
                                                  node_column_source="org:group",
                                                  node_column_target="org:group",
                                                  edge_column="concept:name",
                                                  performance=False)

pm4py.save_vis_network_analysis(frequency_edges, variant="frequency", edge_threshold=10, file_path=SRC_DIR / 'Images' / 'OCPM/3_SNA_.svg')

parsing log, completed traces ::   0%|          | 0/1434 [00:00<?, ?it/s]

<br></br>
## <b>9. OC-DFG discovery</b>
Object-centric directly-follows multigraphs are a composition of directly-follows graphs for the single object type, which can be annotated with different metrics considering the entities of an object-centric event log (i.e., events, unique objects, total objects).

In [25]:
# frequency-based 
ocel = pm4py.read_ocel(str(SRC_DIR / 'Datasets' / 'Example' / 'Orders' / 'example_log.jsonocel'))
ocdfg = pm4py.discover_ocdfg(ocel)
pm4py.save_vis_ocdfg(ocdfg, file_path=SRC_DIR / 'Images' / 'OCPM' / '4_DFG.svg')

In [26]:
# performance-based
ocel = pm4py.read_ocel(str(SRC_DIR / 'Datasets' / 'Example' / 'Orders' / 'example_log.jsonocel'))
ocdfg = pm4py.discover_ocdfg(ocel)
pm4py.save_vis_ocdfg(ocdfg, annotation="performance", performance_aggregation="median", file_path=SRC_DIR / 'Images' / 'OCPM' / '5_DFG.svg')

<br></br>
## <b>10. OC-PN discovery</b>
Object-centric Petri Nets (OC-PN) are formal models, discovered on top of the object-centric event logs, using an underlying process discovery algorithm (such as the Inductive Miner). They have been described in the scientific paper:

van der Aalst, Wil MP, and Alessandro Berti. "Discovering object-centric Petri nets." Fundamenta informaticae 175.1-4 (2020): 1-40.

In [50]:
ocel = pm4py.read_ocel(str(SRC_DIR / 'Datasets' / 'Example/Orders/example_log.jsonocel'))
model = pm4py.discover_oc_petri_net(ocel)
pm4py.save_vis_ocpn(model, file_path=SRC_DIR / 'Images' / 'OCPM/6_PN.svg')

In [51]:
ocel = pm4py.read_ocel(str(SRC_DIR / 'Datasets' / 'Example/Orders/example_log.jsonocel'))
graph = object_codeath_graph.apply(ocel)
graph

{('i1', 'i2'), ('i3', 'i4'), ('i3', 'i7'), ('i4', 'i7'), ('i8', 'i9')}

<br></br><br></br>

## <b>Resources</b>
* https://pm4py.fit.fraunhofer.de/documentation#object-centric-event-logs